In [1]:
import os
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
import re

%matplotlib inline
pd.options.display.max_rows = 150
pd.options.display.max_colwidth = 1000

In [2]:
if("reload" not in globals()):
    reload = True
    
if(reload):
    db = pd.read_excel('recipeInfo.xls',encoding='utf8')
    
    db2 = db.dropna(subset=['ingredients_bag-of-words'],axis=0)
    
    reload = False

db_recipes = db2[['title','ingredients_list']].copy(deep=True)
db_recipes['title'] = db_recipes['title'].str.lower()
db_recipes['ingredients_list'] = db_recipes['ingredients_list'].str.lower()
db_recipes.head()

,title,ingredients_list
0,easy light chocolate milkshake recipe,"put one half cup of milk, 4 tablespoons of chocolate syrup, vanilla extract and ice to taste in whatever your using to blend it. blend until satisfied with the texture and serve.|now what are you waiting for?|go on! enjoy!"
1,lamb stew recipe : : recipes : food network,"5 yellow onions|2 turnips|5 carrots|1 stalk fennel|1 cup cooking oil, divided|1 1/2 gallons chicken stock|1/2 head garlic|1/4 gallon orange juice|1/4 gallon apple cider|1/4 cup coriander seeds|1/8 cup fennel seeds|1 star anise|2 tablespoons cumin seeds|1/8 cup orange zest|1 teaspoon salt|1/2 tablespoon freshly ground black pepper|1 leg of lamb|3 cups dry white wine|salt and freshly ground pepper|your choice fresh herbs, minced, to taste"
2,chocolate bread pudding recipe : paula deen : recipes : food network,"1 (1-pound) loaf french or italian bread, cubed|3 cups milk|1/4 cup heavy cream|1/2 cup coffee flavored liqueur|1 cup sugar|1 cup packed light brown sugar|1/4 cup cocoa powder|1 tablespoon vanilla extract|2 teaspoons almond extract|1 1/2 teaspoon cinnamon|6 eggs, lightly beaten|8 ounces semisweet chocolate, grated|whipped cream (optional)"
3,snowball cookies ii recipe,1/2 cup powdered sugar|1/3 cup butter or margarine|2 cups bisquick baking mix|1/2 cup chopped nuts|1 teaspoon almond extract|2 tablespoons water|powdered sugar
4,old fashioned butterscotch pie recipe #31698,1 1/2 cup brown sugar|1 cup water|3 eggs|4 tablespoons flour|1 cup milk|4 tablespoons butter|1 teaspoon vanilla extract|1/4 teaspoon salt|1/2 teaspoon cream of tartar|4 tablespoons granulated or powdered sugar


In [4]:
db_foods = pd.read_csv('foods_modif.csv',encoding='latin1')
db_foods['name'] = db_foods['name'].str.lower()
db_foods['food_group'] = db_foods['food_group'].str.lower()

#sort by length
db_foods.index = db_foods['name'].str.len()
db_foods = db_foods.sort_index(ascending=False).reset_index(drop=True)

# create foods list
foods_list = db_foods['name'].tolist()

db_foods.head()

,name,food_group
0,italian sweet red pepper,vegetables
1,greenland halibut/turbot,aquatic foods
2,half-highbush blueberry,fruits
3,arctic ground squirrel,animal foods
4,chinese water chestnut,vegetables


In [7]:
non_foods_list = ['teaspoon','teas.','steam','instead']

def match_foods(x):
    ret_list = []
    x_copy = x + ' '
    for non_food in non_foods_list:
        x_copy = x_copy.replace(non_food,'')
    for food in foods_list:
        if ((food+' ') in x_copy or (food+',') in x_copy or
        (food+'|') in x_copy or (food+'s') in x_copy):
            ret_list.append(food)
            x_copy = x_copy.replace(food,'')
    return ret_list

db_recipes['ingredients'] = db_recipes['ingredients_list'].apply(match_foods)

In [8]:
db_recipes.sample(n=1)

,title,ingredients_list,ingredients
42048,exquisite pizza sauce recipe,1 (6 ounce) can tomato paste|6 fluid ounces warm water (110 degrees f/45 degrees c)|3 tablespoons grated parmesan cheese|1 teaspoon minced garlic|2 tablespoons honey|1 teaspoon anchovy paste (optional)|3/4 teaspoon onion powder|1/4 teaspoon dried oregano|1/4 teaspoon dried marjoram|1/4 teaspoon dried basil|1/4 teaspoon ground black pepper|1/8 teaspoon cayenne pepper|1/8 teaspoon dried red pepper flakes|salt to taste,"[black pepper, red pepper, parmesan, oregano, anchovy, garlic, pepper, tomato, cheese, honey, onion, water, salt]"


In [48]:
def search_food(food):
    return db_recipes[db_recipes['ingredients'].apply(lambda x: food in x)]

search_food('tea')

,title,ingredients_list,ingredients
686,sweet tea pie recipe : : recipes : food network,"6 ounces cream cheese, softened|1 cup (2 sticks) unsalted butter, softened|2 1/2 cups unbleached all-purpose flour|1 cup (2 sticks) unsalted butter, softened|2 cups sugar|8 large egg yolks|3/4 cup strong steeped orange pekoe tea, cooled|1 teaspoon grated lemon zest|1 tablespoon fresh lemon juice|2 tablespoons unbleached all-purpose flour|1 1/2 teaspoons cornmeal|1/2 teaspoon salt","[cream cheese, orange, butter, lemon, flour, sugar, corn, salt, tea, egg]"
1085,iced tea ii recipe,8 cups water|3 orange pekoe tea bags|3/4 cup white sugar|1/2 cup lemon juice,"[orange, lemon, water, sugar, tea]"
1127,tea and cider wassail,"freshly brewed hot tea|cranberry juice|apple cider or apple juice|orange juice|sugar|lemon juice|stick cinnamon, broken|whole cloves|apple slices (optional)","[apple cider, cranberry, cinnamon, orange, cloves, apple, lemon, sugar, tea]"
1151,brandy slush i recipe,5 cups boiling water|4 (2 g) bags green tea|1 cup white sugar|1 (16 ounce) can frozen lemonade concentrate|1 (16 ounce) can frozen orange juice concentrate|2 cups brandy|1 (2 liter) bottle lemon-lime flavored carbonated beverage,"[orange, lemon, water, sugar, ling, lime, tea]"
1292,pork chops with mushroom sauce recipe,"1 tablespoon salt|1/4 teaspoon paprika|1/4 teaspoon fresh ground black pepper|4 pork chops (originally called for ""pork rib chops"" i used boneless center cut chops)|2 tablespoons olive oil|1 tablespoon butter (i actually used pam spray for oil and butter)|1 lemon, cut into wedges|1 tablespoon butter|1 small onion, diced|2 cloves garlic, minced|1 1/2 lbs button mushrooms, sliced 1/2 inch thick|1 teaspoon fresh thyme|1/2 teaspoon lemon, zest of (i used way more)|1/2 cup white wine (had no wine, used sherry)|2 cups chicken broth (i only used about a cup)|1 cup heavy cream (i used about 1/2 cup land o'lakes fat free half & half)|1/2 teaspoon salt (didn't use)|1/2 teaspoon fresh ground black pepper (used a dash of cayenne instead)","[black pepper, olive oil, mushrooms, chicken, paprika, garlic, cloves, sherry, butter, onion, lemon, cream, thyme, pork, wine, salt, gin, oil, tea]"
1329,clean shower substitute recipe,"1 teaspoon borax|1 teaspoon washing soda|1 tablespoon liquid soap|1/2 cup vinegar|3 -4 drops , the essential oil of your choice (i use tea tree oil)","[vinegar, oil, tea]"
2688,low cal chai tea latte recipe,"bigelow decaffeinated spiced chai tea, 1 bag|water, 2 cups of 8 fl oz (16 fl oz)|coffee-mate sugar free vanilla caramel powder creamer, 1 tbsp|sweet 'n low|cool whip free, 1 tbsp|ground cinnamon, 1/2 tsp","[cinnamon, vanilla, coffee, water, cream, sugar, tea]"
2944,thai iced tea recipe : emeril lagasse : recipes : food network,6 cups water|1 cup thai tea|3/4 cup sugar|6 tablespoons cream|6 tablespoons condensed milk,"[condensed milk, water, cream, sugar, tea]"
2999,neely's peach iced tea recipe : patrick and gina neely : recipes : food network,"8 bags english breakfast tea|1 bag hibiscus tea|8 cups water|1 (16-ounce) bag sliced frozen peaches, thawed, divided|1 (12-ounce) can peach nectar, chilled|1/4 cup peach liquor|fresh mint, for garnish","[hibiscus tea, liquor, peach, water, mint, tea]"
3323,instant russian tea recipe,2 cups orange-flavored drink mix (e.g. tang)|2 cups white sugar|1/4 cup instant tea powder|3/4 cup lemon-flavored instant tea powder|1 teaspoon ground cinnamon|1 teaspoon ground cloves,"[cinnamon, orange, cloves, lemon, sugar, tea]"
